In [1]:
function parse_and_store_memory(assembly::Vector{String}, memory::Array{Int,2})
    current_row = 1

    for line in assembly
        parts = split(line, ':')
        if length(parts) == 2
            variable_name = strip(parts[1])
            data_values = strip(parts[2])

            if occursin(".word", variable_name)
                values = parse.(Int, split(data_values))
                if current_row <= size(memory, 1)
                    memory[current_row, 1:length(values)] .= values
                    current_row += 1
                else
                    println("Error: Insufficient memory rows for .word directive.")
                end
            elseif occursin(".string", variable_name)
                # Extract string content between double quotes
                string_content = match(r"\"(.*)\"", data_values).captures[1]
                # Convert each character to its ASCII value and store in memory
                values = [Int(c) for c in string_content]

                if current_row <= size(memory, 1)
                    memory[current_row, 1:length(values)] .= values
                    current_row += 1
                else
                    println("Error: Insufficient memory rows for .string directive.")
                end
            end
        end
    end
end

parse_and_store_memory (generic function with 1 method)

In [2]:
assembly_code = ["array: .word 10 20 30", "str: .string \"Hello\"", ""]
memory_array = zeros(Int, 1024, 4)
parse_and_store_memory(assembly_code, memory_array)

In [4]:
# println(memory_array)

In [35]:
function remove_commas(input_string::AbstractString)::AbstractString
    return replace(input_string, "," => " ")
end

function remove_comments(input_string::AbstractString)::AbstractString
    return replace(input_string, r"#.*" => "")
end

function remove_parentheses(input_string::AbstractString)::AbstractString
    return replace(input_string, r"[()]" => " ")
end

function add_spaces(input_string::AbstractString)::AbstractString
    return replace(input_string, r"(\w)([x\d])" => s"\1 \2")
end

add_spaces (generic function with 1 method)

In [36]:
function sanitize(raw_line::AbstractString)::AbstractString
    modified_line = replace(raw_line, r"\b\d+\b" => x -> string(parse(Int, x)))
    modified_line = remove_parentheses(modified_line)
    modified_line = remove_comments(modified_line)
    modified_line = remove_commas(modified_line)
    modified_line = strip(modified_line)
    return modified_line
end

sanitize (generic function with 1 method)

extract_literals (generic function with 1 method)

In [98]:
function extract_literals(assembly::Vector{String})
    literals = []

    for line in assembly
        sanitized_line = sanitize(line)
        parts = split(sanitized_line, ':')

        if length(parts) == 2
            variable_name = strip(parts[1])
            data_values = strip(parts[2])
            
            if occursin(".word", line[1])  # Check if the line contains .word
                # Extract values after .word
                values = parse.(Int, split(data_values))
                append!(literals, values)
            elseif occursin(".string", variable_name)
                # Extract string content between double quotes
                string_content = match(r"\"(.*)\"", data_values)
                if string_content !== nothing
                    append!(literals, [Int(c) for c in string_content.captures[1]])
                else
                    println("Error: Incorrect .string format in line: $line")
                end
            end
        end
    end

    return literals
end

extract_literals (generic function with 1 method)

In [99]:
# Example usage:
assembly_code = ["array: .word 10 20 30", "str: .string \"Hello\"", ""]
literals = extract_literals(assembly_code)

LoadError: MethodError: no method matching occursin(::String, ::Char)

[0mClosest candidates are:
[0m  occursin(::Any)
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4msearch.jl:724[24m[39m
[0m  occursin(::Union{AbstractChar, AbstractString}, [91m::AbstractString[39m)
[0m[90m   @[39m [90mBase[39m [90mstrings/[39m[90m[4msearch.jl:699[24m[39m


In [67]:
println(literals)

Any[]


In [68]:
str = "Hello"

"Hello"

In [42]:
println(str)

Hello


In [103]:
assembly_code = ["array: .word 10 20 30", "str: .string \"Hello\"", ""]

function extract_literals(code)
    literals = []
    for line in code
        # Match numeric literals
        numeric_literals = matchall(r"\b\d+\b", line)
        append!(literals, parse.(Int, numeric_literals))

        # Match string literals
        string_literals = matchall(r"*", line)
        append!(literals, string_literals)
    end
    return literals
end

literals = extract_literals(assembly_code)
println("Literals: ", literals)


LoadError: LoadError: PCRE compilation error: quantifier does not follow a repeatable item at offset 0
in expression starting at In[103]:11

In [1]:
function int_to_binary_5bits(x::Int)::AbstractString
    return lpad(string(x, base=2), 5, "0")
end

int_to_binary_5bits (generic function with 1 method)

In [5]:
x = 18

18

In [7]:
a=int_to_binary_5bits(x)

"10010"

In [10]:
reverse(a[1:3])

"001"

In [15]:
function hex_to_32binary(x::AbstractString)::AbstractString
    return lpad(string(hex_to_int(x), base=2), 32, "0")
end

hex_to_32binary (generic function with 1 method)

In [14]:
function hex_to_int(x::AbstractString)::Int
    return parse(Int, x, base=16)
end

hex_to_int (generic function with 1 method)

In [42]:
a="00309463"

"00309463"

In [43]:
hex_to_32binary(a)

"00000000001100001001010001100011"

In [44]:
a=23423

23423

In [32]:
function int_to_binary_12bits(x::Int)::AbstractString
    return lpad(string(x, base=2), 12, "0")
end

int_to_binary_12bits (generic function with 1 method)

In [33]:
str=int_to_binary_12bits(a)

"101101101111111"

In [37]:
string(str[4])

"1"

In [39]:
# Function to convert string to binary
function string_to_binary(input_string::AbstractString)
    binary_string = join([string(UInt8(c), base=2, pad=8) for c in input_string])
    return binary_string
end

# Example usage
input_string = "Hello, World!"
binary_result = string_to_binary(input_string)
println("Binary representation of '$input_string': $binary_result")


Binary representation of 'Hello, World!': 01001000011001010110110001101100011011110010110000100000010101110110111101110010011011000110010000100001


In [45]:
# Original list of strings
original_strings = ["li x1 5", "li x2 5", "beq x1 x2 exit", "li x3 10", "", "exit", "li x3 20"]

# Remove empty strings
filtered_strings = filter(x -> !isempty(x), original_strings)

# Print the result
println(filtered_strings)


["li x1 5", "li x2 5", "beq x1 x2 exit", "li x3 10", "exit", "li x3 20"]


In [46]:
5 * 6

30